In [4]:
import torch
import sys
import gc

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CODECONTESTS_COT_CONFIG, SMOL_LM_135M

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
cfg = CODECONTESTS_COT_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
wrapper = TrainerWrapper(cfg)

2025-01-07 20:57:11.289 | INFO     | trl_wrapper.trainer_wrapper:__init__:176 - Adding special tokens: ['<thought>', '</thought>', '<solution>', '</solution>']


In [ ]:
wrapper.init_model()

In [6]:
wrapper.init_data_module()

2025-01-07 20:57:13.274 | INFO     | model.utils:__init__:110 - Cache dir: ../dataset_caches/conversation_data_module
2025-01-07 20:57:13.274 | INFO     | dataset.conversation:__init__:31 - Cache dir: ../dataset_caches/conversation_data_module//openo1_sft_formatted_thoughts_conversations.parquet
2025-01-07 20:57:13.275 | INFO     | model.utils:setup:128 - Loading dataset for stage fit
2025-01-07 20:57:13.275 | INFO     | model.utils:setup:131 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module//openo1_sft_formatted_thoughts_conversations.parquet, using cache: False
2025-01-07 20:57:13.276 | INFO     | dataset.conversation:load_dataset:36 - Loading dataset
2025-01-07 20:57:13.345 | INFO     | model.utils:setup:139 - Train dataset size: 14445 Val dataset size: 1605
Map:   0%|          | 0/14445 [00:02<?, ? examples/s]


KeyboardInterrupt: 

In [13]:
type(wrapper.tokenizer)

transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast

In [ ]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
# tokenized = wrapper.data_module.tokenize_conversation([first_batch["conversation"]])
for k in first_batch.keys():
    first_batch[k] = torch.as_tensor(first_batch[k], dtype=torch.long)
wrapper.data_module.visualize_sample(first_batch)

In [ ]:
# mock_labels = first_batch["labels"].clone().detach()
# mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
# mock_ids = first_batch["input_ids"].clone().detach()
for batch in wrapper.data_module.val_dataloader():
    print(batch["input_ids"].shape)
    print(batch["labels"].shape)

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.tokenizer

In [ ]:
wrapper.train()